In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Data Augmentation

In [8]:
data_gen_train=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
train_set=data_gen_train.flow_from_directory("train",batch_size=32,target_size=(64,64),class_mode="categorical")
data_gen_val=ImageDataGenerator(rescale=1./255)
val_set=data_gen_val.flow_from_directory("val",batch_size=32,target_size=(64,64),class_mode='categorical')
data_gen_test=ImageDataGenerator(rescale=1./255)
test_set=data_gen_test.flow_from_directory("test",batch_size=32,target_size=(64,64),class_mode='categorical')

Found 1951 images belonging to 4 classes.
Found 253 images belonging to 4 classes.
Found 106 images belonging to 4 classes.


In [4]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Conv2D,MaxPool2D

## Model Creation

In [7]:
cnn=Sequential()
cnn.add(Conv2D(filters=40,kernel_size=3,activation='relu',input_shape=(64,64,3),padding="same"))
cnn.add(MaxPool2D(pool_size=2,strides=2))
cnn.add(Conv2D(filters=40,kernel_size=3,activation='relu',input_shape=(64,64,3),padding="same"))
cnn.add(MaxPool2D(pool_size=2,strides=2))
cnn.add(Flatten())
cnn.add(Dense(units=150,activation='relu',kernel_initializer='he_uniform',input_shape=(64,64)))
cnn.add(Dense(units=80,activation='relu',kernel_initializer='he_uniform'))
cnn.add(Dense(units=4,activation='softmax',kernel_initializer='glorot_uniform'))

## Training 

In [10]:
cnn.compile(optimizer='adam',loss="categorical_crossentropy",metrics=['accuracy'])
cnn.fit(x=train_set,validation_data=val_set,epochs=10,batch_size=32,shuffle=True)

Epoch 1/10
61/61 [==============================] - 175s 3s/step - loss: 1.0581 - accuracy: 0.5428 - val_loss: 0.7589 - val_accuracy: 0.6759
Epoch 2/10
61/61 [==============================] - 197s 3s/step - loss: 0.7482 - accuracy: 0.6981 - val_loss: 0.6989 - val_accuracy: 0.7075
Epoch 3/10
61/61 [==============================] - 200s 3s/step - loss: 0.6726 - accuracy: 0.7309 - val_loss: 0.7096 - val_accuracy: 0.6996
Epoch 4/10
61/61 [==============================] - 222s 4s/step - loss: 0.5943 - accuracy: 0.7704 - val_loss: 0.4930 - val_accuracy: 0.8103
Epoch 5/10
61/61 [==============================] - 237s 4s/step - loss: 0.5184 - accuracy: 0.7950 - val_loss: 0.5284 - val_accuracy: 0.7945
Epoch 6/10
61/61 [==============================] - 235s 4s/step - loss: 0.5173 - accuracy: 0.7929 - val_loss: 0.3928 - val_accuracy: 0.8458
Epoch 7/10
61/61 [==============================] - 244s 4s/step - loss: 0.4402 - accuracy: 0.8216 - val_loss: 0.5616 - val_accuracy: 0.7391
Epoch 8/10
61

## Saving the model

In [13]:
from tensorflow.keras.models import load_model
cnn.save("Model_cotton_disease.h5")
model=load_model("Model_cotton_disease.h5")

In [37]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 64, 64, 40)        1120      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 40)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 40)        14440     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 40)        0         
_________________________________________________________________
flatten (Flatten)            (None, 10240)             0         
_________________________________________________________________
dense (Dense)                (None, 150)               1536150   
_________________________________________________________________
dense_1 (Dense)              (None, 80)               

## Evaluation on Test Data

In [14]:
model.evaluate(test_set)

4/4 [==============================] - 4s 938ms/step - loss: 0.6103 - accuracy: 0.8113


[0.6102585792541504, 0.8113207817077637]

## Testing the model accuracy on individual image inputs

In [46]:
def op(type,name):
    from tensorflow.keras.preprocessing import image
    import numpy as np
    test_img=image.load_img(f"test\{type}\{name}.jpg",target_size=(64,64))
    test_img=image.img_to_array(test_img)
    test_img=test_img/255
    test_img=np.expand_dims(test_img,axis=0)
    result=model.predict(test_img)
    outcome=np.argmax(result)#np.argmax returns the index of max value in that array
    if outcome==0:
        print("Diseased cotton leaf")
    elif outcome==1:
        print("Diseased cotton plant")
    elif outcome==2:
        print("Fresh cotton leaf")
    else:
        print("Fresh cotton plant")

In [47]:
op("diseased cotton plant","dd (41)")

Diseased cotton plant
